# **Parent Document Retriver**

まず、もともとの長い文章（祖父ドキュメント、PDFなど）を、まず親ドキュメントという大きな単位に分割します。そして、その親ドキュメントをさらに小さい単位の子ドキュメント（小チャンク）に分けます。

ユーザーからのクエリは、この子ドキュメント単位で検索され、一致する部分が見つかれば、その子ドキュメントが含まれる親ドキュメント全体が参照される仕組みになっています。つまり、検索の際には祖父ドキュメント全体は直接利用せず、子ドキュメントで判断することで、関連する親ドキュメントを抽出するという効率的な流れになります。

Reference: [Parent Document Retriver](https://python.langchain.com/docs/how_to/parent_document_retriever/)

ここからは[RAG-Evaluation-Dataset-JA](https://huggingface.co/datasets/allganize/RAG-Evaluation-Dataset-JA)をつかって日本語のpdfを元にRAGと評価を行います。

langsmithは次回から使います。

In [1]:
# set apikey
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

import logging

# httpx のログレベルを WARNING 以上にする（INFO メッセージを非表示にする）
logging.getLogger("httpx").setLevel(logging.WARNING)


In [10]:
from langchain_community.document_loaders import PyPDFLoader
file_path = "../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load_and_split()
documents[0:3]

[Document(metadata={'source': '../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf', 'page': 0, 'page_label': '1'}, page_content='スタートアップの力で\n社会課題解決 と経済成長 を加速する\nスタートアップ育成に向けた政府の取組\n2024年9月'),
 Document(metadata={'source': '../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf', 'page': 1, 'page_label': '2'}, page_content='スタートアップとは\n① スタートアップとは、一般に、以下のような企業をいう。\n1. 新しい企業であって、\n2. 新しい技術やビジネスモデル（イノベーション）を有し、\n3. 急成長を目指す企業\n② スタートアップの意義\n\uf070 スタートアップは、経済成長のドライバー。将来の所得や財政を支える新たな担い手。\n\uf070 スタートアップは、雇用創出にも大きな役割。\n\uf070 スタートアップは、新たな社会課題を解決する主体としても重要。\n■ 日本（TOPIX）と米国（S&P）における直近10年間の株式市場のパフォーマンスの推移*¹\n2出所：ダイヤモンドオンライン「『GAFAM』除けば日米の株価成長は同じ？企業再編から考える米国経済の強さ」\n*１：2013年９月の各終値を100とおいた場合の騰落率。'),
 Document(metadata={'source': '../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf', 'page': 2, 'page_label': '3'}, page_content='社会課題解決・社会貢献の担い手としてのスタートアップ\n\uf070 スタートアップによる社会課題解決・社会貢献の事例\n新型コロナワクチンの開発\n断水中の被災地への\nシャワー、手洗い設備の提供\nWOTA

In [11]:
# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# split pages content
from langchain_text_splitters import RecursiveCharacterTextSplitter

# create the parent documents - The big chunks
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=2000)
# create the child documents - The small chunks
child_splitter = RecursiveCharacterTextSplitter(chunk_size=400)

# The storage layer for the parent chunks
from langchain.storage import InMemoryStore
store = InMemoryStore()

# load vectorstore
from langchain_chroma import Chroma
vectorstore = Chroma(collection_name="split_parents", embedding_function=embeddings)

In [12]:
# create retriever
from langchain.retrievers import ParentDocumentRetriever
retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter
)

# add documents to vectorstore
retriever.add_documents(documents)

In [13]:
# create llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI() # default gpt-turbo-3.5 

# create document chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Set prompt template
template = """"
You are a helpful assistant that answers questions based on the following context
Context: {context}

Question {input}

Answer:

"""

# Set prompt
prompt = ChatPromptTemplate.from_template(template)

# Set RAG pipeline
rag_chain = (
    {"context": retriever, "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
import pandas as pd
from datasets import load_dataset

# === データの読み込みとフィルタリング ===
test_data = load_dataset("allganize/RAG-Evaluation-Dataset-JA")["test"]

tgt_name = file_path.split("file_name=")[-1]  # どこかで定義済みの file_path
print(f"{tgt_name=}")

# 評価対象データだけを抽出
filtered_data = test_data.filter(lambda x: x["target_file_name"] == tgt_name)

# DataFrame 化して、全体の件数を確認
eval_df = filtered_data.to_pandas()
print(f"Length of eval_df: {len(eval_df)}")

# 必要な列だけ取り出す (たとえば 質問と正解回答)
eval_df = eval_df[["question", "target_answer"]]
print(eval_df.head(2))

# === RAG チェーンを利用して回答と文脈を取得する ===
p_data = []  # 各サンプルごとの評価用データを貯めるリスト

for i, row in eval_df.iterrows():
    q = row["question"]
    t = row["target_answer"]
    
    # RAG チェーンを実行して回答を取得するのだー
    # rag_chain や retriever はすでに定義済みの前提
    response_text = rag_chain.invoke(q)
    context_docs = retriever.invoke(q)
    
    p_data.append({
        "question": q,
        "target_answer": t,
        "response": response_text,
        "context": context_docs
    })

# === RAG 実行結果をまとめた新しい DataFrame を作成 ===
rag_eval_df = pd.DataFrame(p_data)
print(rag_eval_df.head())

tgt_name='kaisetsushiryou_2024.pdf'
Length of eval_df: 7
                                            question  \
0  オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...   
1  イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...   

                                       target_answer  
0  オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...  
1  イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...  
                                            question  \
0  オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...   
1  イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...   
2                         「カーブアウト加速等支援事業」の主な目的は何ですか？   
3           グローバル・アクセラレーション・ハブの拠点がある北米の都市を全て教えてください。   
4  産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立したファンドについて、資...   

                                       target_answer  \
0  オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...   
1  イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...   
2  事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...   
3  グローバル・アクセラレーション・ハブの北米の拠点は、ボストン、ニューヨーク、シカゴ、オース

In [22]:
import pandas as pd
from langchain.evaluation import load_evaluator
from langchain.evaluation import EvaluatorType

relevance_evaluator = load_evaluator(
    EvaluatorType.QA,
    criteria="relevance",
    model="gpt-4",
    model_kwargs={"temperature": 0}
)

accuracy_evaluator = load_evaluator(
    EvaluatorType.QA,
    criteria="accuracy",
    model="gpt-4",
    model_kwargs={"temperature": 0}
)

def evaluate_response(df):
    """
    与えられたデータリストに対する Relevance と Accuracy の評価を行い、結果を DataFrame で返します。
    """
    def _flatten_context(context_val):
        if isinstance(context_val, list):
            return "\n".join([item.page_content if hasattr(item, "page_content") else str(item) for item in context_val])
        return str(context_val)
    
    df["ContextText"] = df["context"].apply(lambda x: _flatten_context(x))

    def _evaluate_row(row):
        relevance_score = relevance_evaluator.evaluate_strings(
            prediction=row["response"],
            input=row["question"],
            reference=row["ContextText"]
        )
        accuracy_score = accuracy_evaluator.evaluate_strings(
            prediction=row["response"],
            input=row["question"],
            reference=row["target_answer"]
        )
        return relevance_score, accuracy_score
    
    df[["Relevance", "Accuracy"]] = df.apply(lambda row: pd.Series(_evaluate_row(row)), axis=1)
    
    df["RelevanceScore"] = df["Relevance"].apply(lambda x: 1 if x["value"].upper() == "CORRECT" else 0)
    df["AccuracyScore"] = df["Accuracy"].apply(lambda x: 1 if x["value"].upper() == "CORRECT" else 0)
    
    # 平均スコアを計算
    avg_relevance = df["RelevanceScore"].mean()
    avg_accuracy = df["AccuracyScore"].mean()
    
    print(f"Average Relevance: {avg_relevance:.2f}")
    print(f"Average Accuracy: {avg_accuracy:.2f}")

    return df

In [23]:
results = evaluate_response(rag_eval_df)
results # 3行目でincorrectが出ている

Average Relevance: 0.86
Average Accuracy: 0.71


,question,target_answer,response,context,ContextText,Relevance,Accuracy,RelevanceScore,AccuracyScore
0,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...,オープンイノベーション促進税制において、新規発行株式と発行済株式の場合で税制優遇措置は異なり...,[page_content=' 国内事業会社又はその国内CVCが、オープンイノベーションに..., 国内事業会社又はその国内CVCが、オープンイノベーションにより新事業開拓・生産性向上を図...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
1,イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...,イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...,控除対象となる研究開発活動に関しては、国内で企業が自ら行った知的財産（特許権やAI関連のプロ...,[page_content=' 我が国のイノベーション拠点の立地競争力を強化する観点から、..., 我が国のイノベーション拠点の立地競争力を強化する観点から、海外と比べて遜色ない事業環境の...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
2,「カーブアウト加速等支援事業」の主な目的は何ですか？,事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...,「カーブアウト加速等支援事業」の主な目的は、事業会社に蓄積されている技術を活用し、新たな会社...,[page_content=' 日本企業では、研究開発により得た技術であって、事業化されな..., 日本企業では、研究開発により得た技術であって、事業化されないものの多くが消滅しており、研...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
3,グローバル・アクセラレーション・ハブの拠点がある北米の都市を全て教えてください。,グローバル・アクセラレーション・ハブの北米の拠点は、ボストン、ニューヨーク、シカゴ、オーステ...,北米の都市では、以下の都市にグローバル・アクセラレーション・ハブの拠点があります。\n\n1...,[page_content=' 独立行政法人日本貿易振興機構（JETRO）\nは、世界８地..., 独立行政法人日本貿易振興機構（JETRO）\nは、世界８地域・３０都市において、現地\n...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
4,産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立したファンドについて、資...,産業革新投資機構(JIC)は子会社であるベンチャー・グロース・インベストメンツ(VGI)を通...,産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立した２号ファンドの資金規...,[page_content=' 2023年１月、産業革新投資機構（JIC）の子会社であるベ..., 2023年１月、産業革新投資機構（JIC）の子会社であるベンチャー・グロース・インベスト...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
5,スタートアップ支援資金と挑戦支援資本強化特別貸付の融資限度額と返済期間の違いに加えて、要件や...,スタートアップ支援資金は融資限度額が20億円で直接貸付、返済期間は20年以内、要件はJVCA...,スタートアップ支援資金と挑戦支援資本強化特別貸付の融資限度額と返済期間について、以下のような...,[page_content=' スタートアップの創業等を促進するため、日本政策金融公庫等に..., スタートアップの創業等を促進するため、日本政策金融公庫等において、創業等関連の融資・保証...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",1,0
6,宇宙戦略基金の設立と関連する技術開発テーマの具体的な支援分野について説明し、各分野間でどのよ...,宇宙戦略基金は、民間企業や大学、スタートアップ、国立研究機関に対して10年間にわたる研究開発...,宇宙戦略基金は、宇宙技術戦略に基づいて我が国が推進すべき技術開発テーマを設定し、それに沿って...,[page_content='「宇宙戦略基金」\n■ 概要\n■ スキーム・要件\n■ 実績...,「宇宙戦略基金」\n■ 概要\n■ スキーム・要件\n■ 実績・アピールポイント\n予算額\...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
